In [13]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import glob
import os
import shutil
import pyodbc

def getAccounts():
    """ Function that querys db to get list of accounts
    Retunrs: list of accounts
    """
    conn_str = (
    r"DRIVER={SQL Server};"
    r"SERVER=PRODSQL\MSSQL2014BI;"
    r"DATABASE=CMaster;"
    r"Trusted_Connection=yes;"
    )

    # create Connection and Cursor objects
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    sql = "SELECT DISTINCT Top 1 ACNum FROM Cmaster.dbo.RFQMaster WHERE UtilityID = '17' AND StatusID <14 \
            EXCEPT SELECT DISTINCT ACNum FROM CMaster.dbo.ICAPTags WHERE PeakYr = '2019'" #sql query to get list of accounts with no peakyr 2019 tags
  
    data = pd.read_sql(sql, conn) #takes results into a pandas df
    #accountFile.head()
   
    data['ACNum'] = data['ACNum'].str.lstrip('0')
    conn.close() 
    
    global accountList
    accountList = data['ACNum'].tolist()
    
    return accountList


def chunkList (accountList):
    """
    Chunks list of accounts into text files of 10 
    Parameters:
    accountList: List of accounts created in previous function 
    
    """
    chunks = [accountList[x:x + 10] for x in range(0, len(accountList), 10)
          ]  #splits list into sublists of 10

    for i in chunks:
        with open(
                os.path.join('X:/AAA Database/Databases/Quotes/ICAP/TextFiles/',
                             'cmpAccounts{}.txt'.format(i)), 'w') as output:
            for row in i:
                output.write(str(row) + '\n')
                
def getTags():
    """
    initializes webdriver, accesses CMP website and retrives 2020 tags
    """
    options = webdriver.ChromeOptions()
    options.add_argument('headless') #makes it so no browser appears
    options.add_experimental_option("prefs", {
      "download.default_directory": r"X:\AAA Database\Databases\Quotes\ICAP\Download"}) #Change default downlaod directory so dont have to move 
    
    driver = webdriver.Chrome(options = options)
    
    driver.get('https://sso.cmpco.com/ESMSSO/SupplierLogon.aspx')

    username = driver.find_element_by_name(
        "ctl00$appContent$txtUserID")  #Locates Username field
    password = driver.find_element_by_name(
        'ctl00$appContent$Password')  #Locates PW field

    username.send_keys("AMBaker")  #enters username
    password.send_keys("EnergySupply6$$")  #enters PW need to change when password is changed

    driver.find_element_by_xpath(
        '//*[@id="SupplierLogon"]/div[3]/fieldset/table/tbody/tr/td[1]/table/tbody/tr[8]/td/input[1]'
    ).click()  ##Clicks I agree and logs in

    driver.implicitly_wait(
        5)  #waits 5 seconds for page to load, update to wait for element

    driver.find_element_by_xpath(
        '//*[@id="main"]/div/table/tbody/tr/td/p[1]/input').click(
        )  #chooses capaacity data

    Capability_period = driver.find_element_by_xpath(
        '//*[@id="CapPeriod"]/option[2]').click(
        )  #finds 2020 tags and clicks option may need to be updated in the future

    File_Format = driver.find_element_by_xpath(
        '//*[@id="FileFormat_false"]').click(
        )  #Selects csv files instead of xml

    fileList = glob.glob(
        'X:/AAA Database/Databases/Quotes/ICAP/TextFiles/*.txt'
    )  #creates list of text files that where created in Chunk List

    for i in fileList:
        Account_file = driver.find_element_by_xpath(
            '//*[@id="txtFile"]'
        )  #Locates account file field //*[@id="txtFile"]
        Account_file.send_keys(i)  #enters account file
        driver.find_element_by_xpath(
            '//*[@id="onclickPC_2"]').click()  #runs file query

        try:
            WebDriverWait(driver, 15).until(EC.alert_is_present(
            ))  #waits for alert to appear to click on 'OK'

            alert = driver.switch_to.alert  #finds alert
            alert.accept()  #accepts alert
            print("alert accepted")
        except TimeoutException:
            driver.close(
            )  #time's out after 15 seconds of waiting, closses web browser
            print("No Alert")

    for y in fileList: 
        shutil.move(y, r'X:/AAA Database/Databases/Quotes/ICAP/Archive/TextFiles')  #moves text file to archive

    global capFiles 
    capFiles = glob.glob('X:/AAA Database/Databases/Quotes/ICAP/Download/*.csv')

    driver.close()  #closes webDriver
    return capFiles


def importTags(capFiles):
    """
    Imports downloaded files and formats them into a df 
    
    Parameters:
    CapFiles: List of downloaded files
    
    Returns:
    masterDF: Dataframe of all tags and account numbers. Tags in KW 
    """
    global masterDF
    masterDF = pd.DataFrame(columns=['Account Number', 'Tag Value in MW'])
    for x in capFiles:
        capFile = pd.read_csv(x)
        masterDF = masterDF.append(capFile, sort = True)

    masterDF = masterDF.reset_index()

    for x in capFiles:
        shutil.move(x, r'X:/AAA Database/Databases/Quotes/ICAP/Archive/CSV')

    masterDF = masterDF.drop(columns=['index'])
    masterDF['Tag Value in MW'] = pd.to_numeric(masterDF['Tag Value in MW'],
                                                errors='coerce')
    masterDF['TagsKWH'] = 1000 * masterDF['Tag Value in MW']
    masterDF = masterDF.drop(columns = 'Tag Value in MW')

    return masterDF

getAccounts()
chunkList(accountList)
getTags()
importTags(capFiles)

masterDF.to_csv (r'X:/AAA Database/Databases/Quotes/ICAP/Output/Output.csv', index = False)
print("done")

alert accepted
done


SELECT DISTINCT a.Acnum
FROM Cmaster.dbo.RFQMaster AS a
LEFT JOIN (SELECT b.TrDate, b.ICAPtag, b.AcNum, b.UtilityID FROM SQL2017.data.icap.NEISO_Tags  AS b WHERE b.UtilityID = 17 AND b.TrDate >= '2020-06-01') AS bb 
	ON bb.AcNum = a.AcNum
	AND bb.UtilityID = a.UtilityID
WHERE a.UtilityID = '17'
AND StatusID < '14'
AND bb.ICAPtag IS null